# (나의) 구글 Location History 데이터 지도에 표시하기

-------------------------------

### 지도 라이브러리

In [ ]:
!pip install folium

In [ ]:
import folium

folium.__version__

In [ ]:
import folium

#------------------
# 중심위치 [위도,경도]=[Latitude, longitude] 
#------------------
m = folium.Map(location = [37.549328, 126.8513969],
               zoom_start=16)

# 마커 표시하기
folium.Marker(location = [37.549328, 126.8513969],
              popup = '강서여성인력개발센터', 
              icon=folium.Icon(color='red', icon='star')
).add_to(m)
m

## [실습] (나의) 구글 Location History 데이터 지도에 표시하기


### 1. 구글 Location History 데이터 다운로드
- 구글 크롬 브라우저에 로그인 된 상태에서
- https://takeout.google.com/settings/takeout 사이트로 이동하기
- 모두 체크를 해제시키고 **[위치 기록]** 만 체크표시하고 
- takeout~.zip 파일이 다운로드 된다.

### 2. 필요한 라이브러리 설치하기
설치시 오류가 발생하면 윈도우 명령프롬프트창에서 timezonefinder 설치를 진행해 본다.

In [ ]:
import numpy as np

np.__version__

In [ ]:
!pip install timezonefinder

In [4]:
# 경도, 위도 값으로 시간대 정보 가져오기
from timezonefinder import TimezoneFinder

tzf = TimezoneFinder()
tz = tzf.timezone_at(lng=128, lat=38) # 'Asia/Seoul'
# tz = tzf.timezone_at(lng=13.358, lat=52.5061) # 'Europe/Berlin'
tz

'Europe/Berlin'

### 3.나의 Local History데이터 정보를 딕셔너리타입으로 변환하기

In [6]:
from zipfile import ZipFile
import pandas as pd
import json
from timezonefinder import TimezoneFinder

In [ ]:
# Google Location History takeout file
in_file = ""
out_file = ""

# store all place into this array
place_visits = []

with ZipFile(in_file) as myzip:
    for file in myzip.filelist[:]:
        filename = file.filename

        if "Sematic Location History" in filename:
            # process all file in "Sematic Location History" directory
            history_json = json.load(myzip.open(filename))

            for timeline_object in history_json["timelineObjects"]:

                if "placeVisit" in timeline_object:
                    place_visits_json = timeline_object["placeVisit"]

                    # 위도와 경도 값 없으면 skip
                    if not "location" in place_visits_json or not "latitudeE7" in place_visits_json["location"]:
                        continue

                    place_visit = {
                        "placeId" : place_visits_json["location"]["placeId"],
                        "locationConfidence" : place_visits_json["location"]["locationConfidence"],
                        "startTimestamp" : place_visits_json["duration"]["startTimestamp"],
                        "endTimestamp" : place_visits_json["duration"]["endTimestamp"],
                        "placeVisitImportance" : place_visits_json["placeVisitImportance"],
                        "placeVisitType" : place_visits_json["placeVisitType"],
                        "latitudeE7" : place_visits_json["location"]["latitudeE7"],
                        "longitudeE7" : place_visits_json["location"]["longitudeE7"],
                    }

                    for optional_field in ["centerLatE7", "centerLngE7"]:
                        if optional_field in place_visits_json:
                            place_visit[optional_field] = place_visits_json[optional_field]
                        else:
                            place_visit[optional_field] = None
                    
                    for optional_field in ["name", "address"]:
                        if optional_field in place_visits_json["location"]:
                            place_visit[optional_field] = place_visits_json["location"][optional_field]
                        else:
                            place_visit[optional_field] = None
                    
                    place_visits.append(place_visit)

print(f'레코드 총 개수: {len(place_visits)}')
place_visits[:3]

### 4. DataFrame 형태로 변환하기

In [ ]:
# 1.DataFrame 형태로 변환하기
place_visits_df = pd.DataFrame(place_visits)
place_visits_df.head()

# 2.데이터 전처리하기
# convert to datetime type
place_visits_df["startTimestamp"] = pd.to_datetime(place_visits_df["startTimestamp"], format="ISO8601")
place_visits_df["endTimestamp"] = pd.to_datetime(place_visits_df["endTimestamp"], format="ISO8601")

# get geo coordinates as float value
place_visits_df["latitude"] = place_visits_df.latitudeE7/10**7 #(1E7 = 1e7 = 1x10^7 = 10**7)
place_visits_df["longitude"] = place_visits_df.longitudeE7/1e7
place_visits_df["centerLat"] = place_visits_df.centerLatE7/1E7
place_visits_df["centerLng"] = place_visits_df.centerLngE7/1E7

# add timezone based on geo coordinates
tf = TimezoneFinder()
place_visits_df["timezone"] = place_visits_df.apply(lambda row: tf.timezone_at(lng=row.longitude, lat=row.latitude), axis=1)

# convert UTC time to local timezone
place_visits_df['startTimestamp_local'] = place_visits_df.apply(lambda row: row.startTimestamp.tz_convert(row.timezone), axis=1)
place_visits_df['endTimestamp_local'] = place_visits_df.apply(lambda row: row.endTimestamp.tz_convert(row.timezone), axis=1)

# remove TZ info from datetime
place_visits_df['startTimestamp_local'] = pd.to_datetime(place_visits_df['startTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))
place_visits_df['endTimestamp_local'] = pd.to_datetime(place_visits_df['endTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))

# add datetime parts as a separate column to data frame
for datetime_type in [("year", lambda x: x.year), ("month", lambda x: x.month), ("day", lambda x: x.day), ("hour", lambda x: x.hour), ("minute", lambda x: x.minute), ("weekday", lambda x: x.weekday)]:
    for tztype in ["", "_local"]:
        place_visits_df[f"{datetime_type[0]}{tztype}"] = datetime_type[1](place_visits_df[f"startTimestamp{tztype}"].dt)

place_visits_df.drop(columns=["latitudeE7", "longitudeE7", "centerLatE7", "centerLngE7"], inplace=True)
place_visits_df["duration"] = place_visits_df.endTimestamp - place_visits_df.startTimestamp
place_visits_df["duration_minutes"] = place_visits_df.duration.dt.total_seconds()/60

### 5. (나의)Location History 지도에 표시하기

--------------------

THE END